In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk

import json
import csv

import plotly.graph_objects as go

import string
import sys
import random

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import numpy as np
import os

In [2]:
if os.getcwd().split('/')[-1] != 'winogrande_1.1':
    os.chdir('/home/bssachde/Winogrande/winogrande_1.1')

In [3]:
data = []
with open('train.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
D=pd.DataFrame(data)
S=D

In [5]:
S

,qID,sentence,option1,option2,answer
0,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,2
1,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,1
2,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1,"He never comes to my home, but I always go to ...",home,house,1
3,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2,"He never comes to my home, but I always go to ...",home,house,2
4,3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2,"Kyle doesn't wear leg warmers to bed, while Lo...",Kyle,Logan,2
...,...,...,...,...,...
9995,3ZG552ORAM2T6G7V1E3PMORI94N2VA-1,Studying economics was far easier for Megan th...,Megan,Monica,1
9996,3ZLW647WAN9OOW4J129JVRN8D4F32J-1,Lorelei took the carnation out of the jar and ...,jar,vase,1
9997,3ZLW647WAN9OOW4J129JVRN8D4F32J-2,Lorelei took the carnation out of the jar and ...,jar,vase,2
9998,3KA7IJSNW63AP9AVYVN1HP54HO0BP5-2,Felicia only likes dogs for pets while Laura o...,Felicia,Laura,2


In [6]:
#### The y label will be of the form [ 0 1 ] which implies the second option is the answer ####
y = []
for answer_index in range(0,len(S['answer'])):
#     print(S['answer'][answer_index])
    if S['answer'][answer_index]=='2':
        y.append([0,1])
#         y.append(S['option2'][answer_index])
    else:
        y.append([1,0])
#         y.append(S['option1'][answer_index])

In [7]:
### Verify the label format ###
print(len(y))

10000


In [8]:
## Target Dataset size ##
n = 5000

In [9]:
## Load module for test train split
from sklearn.model_selection import train_test_split

## Load set of Linear models
from sklearn.linear_model import LogisticRegression

## Load non-Linear Models
from sklearn import svm

In [10]:
# transformer libraries for fine tune embeddings
from pytorch_transformers import BertPreTrainedModel, RobertaConfig, \
    ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP, RobertaModel
from pytorch_transformers.modeling_roberta import RobertaClassificationHead
from torch.nn import CrossEntropyLoss



In [11]:
## This is for pretraining
# !git clone https://github.com/allenai/winogrande.git

In [12]:
# # os.chdir("winogrande/")
# !export PYTHONPATH=$PYTHONPATH:$(pwd)
# !python "scripts/run_experiment.py" --model_type roberta_mc --model_name_or_path roberta-large --task_name winogrande --do_eval --do_lower_case --data_dir ./data  --max_seq_length 80  --per_gpu_eval_batch_size 4  --per_gpu_train_batch_size 16  --learning_rate 1e-5  --num_train_epochs 3  --output_dir ./output/models/  --do_train  --logging_steps 4752  --save_steps 4750  --seed 42  --data_cache_dir ./output/cache/  --warmup_pct 0.1  --evaluate_during_training

In [13]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)#, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

In [14]:
output_model_file = "models/pytorch_model.bin"
model_state_dict = torch.load(output_model_file) 
model = RobertaModel.from_pretrained('roberta-large',state_dict=model_state_dict)

In [15]:
outputs = model(input_ids)#, masked_lm_labels=input_ids)

In [16]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.']
# outputembeddings = model.get_output_embeddings()
X = []
# len(S['sentence'])
for sentence in sentences:
#     print(sentence)
    input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    X.append(model(input_ids))#, masked_lm_labels=input_ids)

In [17]:
### The model basically returns the top layer embeddings and the pooled layer ###
for element in X:
    print(element[1])

tensor([[ 0.2019,  0.7249,  0.3812,  ..., -0.0937,  0.5541, -0.4825]],
       grad_fn=<TanhBackward>)
tensor([[ 0.2273,  0.7234,  0.4199,  ..., -0.1292,  0.5610, -0.4770]],
       grad_fn=<TanhBackward>)


In [18]:
# ### We are using pickle to save the progress in a file ###
import pickle

# X_prime = []
# ## Check if there was already some progress then continue from there otherwise just start from begining
# if not os.path.isfile("start_pooled.data"):
#     start = 0
# else:
#     with open("start_pooled.data","r") as fileHandle:
#         start = fileHandle.readline()
#         # Scenario if file is there but nothing is in the file
#         if start == '':
#             start = 0
# for index in range(int(start),len(S['sentence'])):
# #     if index==78:
# #         index=index+5
#     if index%50==0:
#         print("Saving the progress for embeddings")
#         with open('listfile_pooled.data', 'a+b') as filehandle:
#             pickle.dump(X_prime, filehandle, pickle.HIGHEST_PROTOCOL)
#         ## Reset the embeddings list X_prime and just keep appending the data in the file
#         X_prime = []
#         ## overwrite the index so far in seprate file
#         with open("start_pooled.data","w") as fileHandle:
#             fileHandle.write(str(index))
#     print(index)
#     input_ids = torch.tensor(tokenizer.encode(S['sentence'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#     X_prime.append(model(input_ids)[1][0])#, masked_lm_labels=input_ids)

# ### We have to save one more time after the loop is over otherwise the last 50 value won't be saved just calculated ###    
# print("Saving the progress for embeddings")
# with open('listfile_pooled.data', 'a+b') as filehandle:
#     pickle.dump(X_prime, filehandle, pickle.HIGHEST_PROTOCOL)


In [19]:
# ### Now we have to get the pooled embeddings for both the options ### 

# # y_emb_option_1 = []
# y_emb_option_2 = []


# ## region ## This is for mainting the position where the code was inturrupted due to any reason what so ever ##
# if not os.path.isfile("start_y_pooled.data"):
#     start = 0
# else:
#     with open("start_y_pooled.data","r") as fileHandle:
#         start = fileHandle.readline()
#         # Scenario if file is there but nothing is in the file
#         if start == '':
#             start = 0
# ## end region 

# # ## getting embedding for option 1
# # for index in range(0,len(S['option1'])):
# #     if index%50==0 and index!=0:
# #         print("Saving the progress for embeddings")
# #         with open('y_label_pooled_option1.data', 'a+b') as filehandle:
# #             pickle.dump(y_emb_option_1, filehandle, pickle.HIGHEST_PROTOCOL)
# #         ## Reset the embeddings list X_prime and just keep appending the data in the file
# #         y_emb_option_1 = []
# #         ## overwrite the index so far in seprate file
# #         with open("start_y_pooled.data","w") as fileHandle:
# #             fileHandle.write(str(index))
# #     print(index)
# #     input_ids = torch.tensor(tokenizer.encode(S['option1'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# # #     print(input_ids)
# #     output_ids = model(input_ids)[1][0]
# #     y_emb_option_1.append(output_ids)#, masked_lm_labels=input_ids)
    
# # print("Saving the progress for embeddings")
# # with open('y_label_pooled_option1.data', 'a+b') as filehandle:
# #     pickle.dump(y_emb_option_1, filehandle, pickle.HIGHEST_PROTOCOL)
# # ## overwrite the index so far in seprate file
# # with open("start_y_pooled.data","w") as fileHandle:
# #     fileHandle.write(str(index))


# ## getting embedding for option 2
# for index in range(int(start),len(S['option2'])):
#     if index%50==0 and index!=0:
#         print("Saving the progress for embeddings")
#         with open('y_label_pooled_option2.data', 'a+b') as filehandle:
#             pickle.dump(y_emb_option_2, filehandle, pickle.HIGHEST_PROTOCOL)
#         ## Reset the embeddings list X_prime and just keep appending the data in the file
#         y_emb_option_1 = []
#         ## overwrite the index so far in seprate file
#         with open("start_y_pooled.data","w") as fileHandle:
#             fileHandle.write(str(index))
#     print(index)
#     input_ids = torch.tensor(tokenizer.encode(S['option2'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# #     print(input_ids)
#     output_ids = model(input_ids)[1][0]
#     y_emb_option_2.append(output_ids)#, masked_lm_labels=input_ids)
    
# print("Saving the progress for embeddings")
# with open('y_label_pooled_option2.data', 'a+b') as filehandle:
#     pickle.dump(y_emb_option_2, filehandle, pickle.HIGHEST_PROTOCOL)
# ## overwrite the index so far in seprate file
# with open("start_y_pooled.data","w") as fileHandle:
#     fileHandle.write(str(index))
        

In [20]:
### verify the Embeddings for options ###
# print(y_emb_option_2)

In [21]:
os.listdir()
os.path.getsize('y_label_pooled_option2.data')

241175722

In [22]:
X = []
y_option1 = []
y_option2 = []

In [23]:
with open('listfile_pooled.data', mode='br') as f:
    while 1:
        try:
            X.append(pickle.load(f))
        except:
            break
with open('y_label_pooled_option1.data', mode='br') as f:
    while 1:
        try:
            y_option1.append(pickle.load(f))
        except:
            break
with open('y_label_pooled_option2.data', mode='br') as f:
    while 1:
        try:
            y_option2.append(pickle.load(f))
        except:
            break
            
### The final features look something like follows:
### <Sentence embeddings> <Option 1> <Sentence embeddings> <Option 2>

In [24]:
flat_list = []
for sublist in X:
    for item in sublist:
        flat_list.append(item.detach().numpy())

X = flat_list[::]
## make a copy for the Sentence embeddings 
X_copy = flat_list[::]            

flat_list = []
for sublist in y_option1:
    for item in sublist:
        flat_list.append(item.detach().numpy())

y_option1 = flat_list[::]

flat_list = []
for sublist in y_option2:
    for item in sublist:
        flat_list.append(item.detach().numpy())

y_option2 = flat_list[:10000]

In [25]:
print(len(X))
print(len(y_option2))
print(S['sentence'][0])
# X_prime = []
# for element in X:
#     X_prime.append(np.asarray(element[0][0]))
X_prime = []
for x1,y1,x2,y2 in zip(X,y_option1,X_copy,y_option2):
#     print(len(x1))
#     print(len(y1))
#     print(len(x2))
#     print(len(y2))
    temp = [*x1,*y1,*x2,*y2]
#     print(len(temp))
    X_prime.append(temp)
    

10000
10000
Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.


In [26]:
# for element in X_prime:
#     print(len(element))
# print(y.shape)
X_temp = np.asarray(X_prime)

In [27]:
X_temp.shape
y_temp = np.asarray(y)
print(y_temp.shape)

(10000, 2)


In [28]:
# y_prime = []
# for element in y:
#     y_prime.append(np.asarray(element[0][0]))

In [29]:
# for element in y_prime:
#     print(element.shape)
    
# y_double_prime = S['answer'][:-50]

In [30]:
# y_double_prime

In [ ]:
from skmultilearn.problem_transform import LabelPowerset

def find_index_of_array(list, array):
    for i in range(len(list)):
        if np.all(list[i]==array):
            return i
# while len(S) > n:
#Dictionary to save the mdoel predictions for every individual elements.
E = dict.fromkeys(S['qID'])
EPrime = dict.fromkeys(S['sentence'])

D = {}
results = {}
# print(D)
m=10
for i in range(0,m):
    ## 90 percent of the data is in the test set
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.90, random_state=42)
    
    ## Train models
    ### Logistic Regression
    lm = LabelPowerset(LogisticRegression(random_state=0,max_iter=1000))
    lm.fit(X_train, y_train)
    
    ### SVM
    rbf_svc = LabelPowerset(svm.SVC(kernel='rbf'))
    rbf_svc.fit(X_train,y_train)
    
    # Predictions
#     for element in X_test:

    predictions = lm.predict(X_test)
    predictions_svm = rbf_svc.predict(X_test)
#     X_test_dict = {}
    for i in range(0,len(X_test)):
#         print(X_test[i])
        index = find_index_of_array(X,X_test[i])
#         print(index)
        if index not in D.keys():
            D[index] = [predictions[i],predictions_svm[i]]
        else:
            D[index].append(predictions[i])
            D[index].append(predictions_svm[i])
        results[index] = y_test[i]
        
        
#predictibility score
P = {}
for key,value in D.items():
#     print(value)
#     print(results[key])
#     print(value.count(results[key])/len(value))
    P[key] = value.count(results[key])/len(value)
# print(P)
#     print(predictions)   

/home/bssachde/.conda/envs/math_deomain2/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning:

elementwise comparison failed; this will raise an error in the future.



In [ ]:
P